In [1]:
import pandas as pd
import altair as alt

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [146]:
cost_breakdown_df = pd.read_csv("cost_breakdown.csv")
customer_feedback_df = pd.read_csv("customer_feedback.csv")
delivery_performance_df = pd.read_csv("delivery_performance.csv")
orders_df = pd.read_csv("orders.csv", keep_default_na=False)
routes_distance_df = pd.read_csv("routes_distance.csv", keep_default_na=False)
vehicle_fleet_df = pd.read_csv("vehicle_fleet.csv")
warehouse_inventory_df = pd.read_csv("warehouse_inventory.csv")

In [93]:
print(f"Duplicates in orders_df: {orders_df['Order_ID'].duplicated().sum()}")
print(f"Duplicates in delivery_performance_df: {delivery_performance_df['Order_ID'].duplicated().sum()}")

Duplicates in orders_df: 0
Duplicates in delivery_performance_df: 0


In [95]:
try:
    orders_df['Order_Date'] = pd.to_datetime(orders_df['Order_Date'])
    warehouse_inventory_df['Last_Restocked_Date'] = pd.to_datetime(warehouse_inventory_df['Last_Restocked_Date'])
    customer_feedback_df['Feedback_Date'] = pd.to_datetime(customer_feedback_df['Feedback_Date'])
    print("Converted 'Order_Date', 'Last_Restocked_Date', and 'Feedback_Date' to datetime objects.")
    print(f"Order_Date type: {orders_df['Order_Date'].dtype}")
except Exception as e:
    print(f"Error converting dates: {e}")

Converted 'Order_Date', 'Last_Restocked_Date', and 'Feedback_Date' to datetime objects.
Order_Date type: datetime64[ns]


In [98]:
print("Missing values in orders_df:\n", orders_df.isnull().sum().loc[lambda x: x > 0])
print("\nMissing values in delivery_performance_df:\n", delivery_performance_df.isnull().sum().loc[lambda x: x > 0])
print("\nMissing values in routes_distance_df:\n", routes_distance_df.isnull().sum().loc[lambda x: x > 0])
print("\nMissing values in warehouse_inventory_df:\n", warehouse_inventory_df.isnull().sum().loc[lambda x: x > 0])
print("\n(No output here means no missing values were found in these files.)")

Missing values in orders_df:
 Special_Handling    153
dtype: int64

Missing values in delivery_performance_df:
 Series([], dtype: int64)

Missing values in routes_distance_df:
 Series([], dtype: int64)

Missing values in warehouse_inventory_df:
 Series([], dtype: int64)

(No output here means no missing values were found in these files.)


In [147]:
print("Renaming 'None' values")
try:
    # For Special_Handling
    orders_df['Special_Handling'] = orders_df['Special_Handling'].replace('None', 'Normal Handling')
    print("Replaced 'None' with 'Normal Handling' in 'Special_Handling'.")
    
    # For Weather_Impact
    routes_distance_df['Weather_Impact'] = routes_distance_df['Weather_Impact'].replace('None', 'Normal Weather')
    print("Replaced 'None' with 'Normal Weather' in 'Weather_Impact'.")

except Exception as e:
    print(f"Error during renaming: {e}")

Renaming 'None' values
Replaced 'None' with 'Normal Handling' in 'Special_Handling'.
Replaced 'None' with 'Normal Weather' in 'Weather_Impact'.


In [148]:
print("Checking for Outliers")
print("\nOrder Value (orders_df):")
print(orders_df['Order_Value_INR'].describe())

print("\nDelivery Days (delivery_performance_df):")
print(delivery_performance_df[['Promised_Delivery_Days', 'Actual_Delivery_Days']].describe())

print("\nRoute Info (routes_distance_df):")
print(routes_distance_df[['Distance_KM', 'Fuel_Consumption_L', 'Toll_Charges_INR', 'Traffic_Delay_Minutes']].describe())


Checking for Outliers

Order Value (orders_df):
count      200.000000
mean      1850.403000
std       4592.326209
min          1.910000
25%        150.040000
50%        500.855000
75%       1371.282500
max      47177.070000
Name: Order_Value_INR, dtype: float64

Delivery Days (delivery_performance_df):
       Promised_Delivery_Days  Actual_Delivery_Days
count              150.000000            150.000000
mean                 4.433333              5.513333
std                  2.630275              3.231009
min                  1.000000              1.000000
25%                  2.250000              3.000000
50%                  4.000000              5.000000
75%                  5.000000              7.000000
max                 10.000000             15.000000

Route Info (routes_distance_df):
       Distance_KM  Fuel_Consumption_L  Toll_Charges_INR  Traffic_Delay_Minutes
count   150.000000          150.000000        150.000000             150.000000
mean   2003.681667          240.57

In [149]:
print("Checking for Categorical Anomalies")
print("\nSpecial Handling (orders_df):")
print(orders_df['Special_Handling'].value_counts())

print("\nPriority (orders_df):")
print(orders_df['Priority'].value_counts())

print("\nCarrier (delivery_performance_df):")
print(delivery_performance_df['Carrier'].value_counts())

print("\nWeather Impact (routes_distance_df):")
print(routes_distance_df['Weather_Impact'].value_counts())

print("Data Cleaning Complete")

Checking for Categorical Anomalies

Special Handling (orders_df):
Special_Handling
Normal Handling           153
Fragile                    23
Temperature_Controlled     13
Hazmat                     11
Name: count, dtype: int64

Priority (orders_df):
Priority
Standard    84
Economy     70
Express     46
Name: count, dtype: int64

Carrier (delivery_performance_df):
Carrier
SpeedyLogistics    41
ReliableExpress    40
QuickShip          31
GlobalTransit      22
EcoDeliver         16
Name: count, dtype: int64

Weather Impact (routes_distance_df):
Weather_Impact
Normal Weather    106
Light_Rain         24
Heavy_Rain         14
Fog                 6
Name: count, dtype: int64
Data Cleaning Complete


Inspecting orders

In [150]:
print(orders_df.info())
print(orders_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          200 non-null    object 
 1   Order_Date        200 non-null    object 
 2   Customer_Segment  200 non-null    object 
 3   Priority          200 non-null    object 
 4   Product_Category  200 non-null    object 
 5   Order_Value_INR   200 non-null    float64
 6   Origin            200 non-null    object 
 7   Destination       200 non-null    object 
 8   Special_Handling  200 non-null    object 
dtypes: float64(1), object(8)
memory usage: 14.2+ KB
None
    Order_ID  Order_Date Customer_Segment  Priority Product_Category  Order_Value_INR     Origin Destination Special_Handling
0  ORD000001  2025-10-09       Individual   Express       Industrial           238.73    Kolkata   Hyderabad  Normal Handling
1  ORD000002  2025-09-29              SMB   Express       Industrial 

Inspecting delivery performance

In [151]:
print(delivery_performance_df.info())
print(delivery_performance_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order_ID                150 non-null    object 
 1   Carrier                 150 non-null    object 
 2   Promised_Delivery_Days  150 non-null    int64  
 3   Actual_Delivery_Days    150 non-null    int64  
 4   Delivery_Status         150 non-null    object 
 5   Quality_Issue           150 non-null    object 
 6   Customer_Rating         150 non-null    int64  
 7   Delivery_Cost_INR       150 non-null    float64
dtypes: float64(1), int64(3), object(4)
memory usage: 9.5+ KB
None
    Order_ID          Carrier  Promised_Delivery_Days  Actual_Delivery_Days   Delivery_Status Quality_Issue  Customer_Rating  Delivery_Cost_INR
0  ORD000001  SpeedyLogistics                       1                     2  Slightly-Delayed       Perfect                3             387.86
1  ORD00000

In [152]:
merged_df = pd.merge(orders_df, delivery_performance_df, on="Order_ID", how="inner")

In [153]:
merged_df = pd.merge(merged_df, routes_distance_df, on="Order_ID", how="inner")

In [154]:
warehouse_inventory_df.rename(columns={'Location': 'Origin'}, inplace=True)

In [155]:
merged_df = pd.merge(merged_df, warehouse_inventory_df, on=["Origin", "Product_Category"], how="left")

In [156]:
print(merged_df.info())
print(merged_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order_ID                150 non-null    object 
 1   Order_Date              150 non-null    object 
 2   Customer_Segment        150 non-null    object 
 3   Priority                150 non-null    object 
 4   Product_Category        150 non-null    object 
 5   Order_Value_INR         150 non-null    float64
 6   Origin                  150 non-null    object 
 7   Destination             150 non-null    object 
 8   Special_Handling        150 non-null    object 
 9   Carrier                 150 non-null    object 
 10  Promised_Delivery_Days  150 non-null    int64  
 11  Actual_Delivery_Days    150 non-null    int64  
 12  Delivery_Status         150 non-null    object 
 13  Quality_Issue           150 non-null    object 
 14  Customer_Rating         150 non-null    in

In [157]:
merged_df['is_delayed'] = (merged_df['Actual_Delivery_Days'] > merged_df['Promised_Delivery_Days']).astype(int)

In [158]:
print(merged_df[['Order_ID', 'Promised_Delivery_Days', 'Actual_Delivery_Days', 'is_delayed']].head())

    Order_ID  Promised_Delivery_Days  Actual_Delivery_Days  is_delayed
0  ORD000001                       1                     2           1
1  ORD000002                       2                     3           1
2  ORD000003                      10                    15           1
3  ORD000004                       5                     5           0
4  ORD000005                       4                     5           1


In [159]:
print(merged_df['is_delayed'].value_counts(normalize=True))

is_delayed
0    0.533333
1    0.466667
Name: proportion, dtype: float64


46.67% Orders are Delayed 

In [160]:
total_orders = len(merged_df)
delayed_orders = merged_df['is_delayed'].sum()
delay_rate = (delayed_orders / total_orders) * 100

In [161]:
print(f"Total Orders Analyzed: {total_orders}")
print(f"Delayed Orders: {delayed_orders}")
print(f"Overall Delay Rate: {delay_rate:.2f}%")

Total Orders Analyzed: 150
Delayed Orders: 70
Overall Delay Rate: 46.67%


In [193]:
status_counts = merged_df['Delivery_Status'].value_counts().reset_index()
status_counts.columns = ['Delivery_Status', 'Count']

In [194]:
total_orders = len(merged_df)
status_counts['Percentage'] = (status_counts['Count'] / total_orders) * 100

In [195]:
print(f"Total Orders Analyzed: {total_orders}\n")
print(status_counts.to_markdown(index=False, floatfmt='.2f'))

Total Orders Analyzed: 150

| Delivery_Status   |   Count |   Percentage |
|:------------------|--------:|-------------:|
| On-Time           |      80 |        53.33 |
| Slightly-Delayed  |      45 |        30.00 |
| Severely-Delayed  |      25 |        16.67 |


In [162]:
priority_delay_df = merged_df.groupby('Priority')['is_delayed'].mean().reset_index()

In [163]:
priority_delay_df = priority_delay_df.rename(columns={'is_delayed': 'Delay_Rate'})
priority_delay_df['Delay_Rate_Percent'] = priority_delay_df['Delay_Rate'] * 100

In [164]:
print(priority_delay_df.to_markdown(index=False, floatfmt=".2f"))

| Priority   |   Delay_Rate |   Delay_Rate_Percent |
|:-----------|-------------:|---------------------:|
| Economy    |         0.45 |                44.90 |
| Express    |         0.50 |                50.00 |
| Standard   |         0.46 |                46.03 |


In [165]:
priority_chart = alt.Chart(priority_delay_df).mark_bar().encode(
    # Use 'Priority' on the x-axis, sort by delay rate (descending)
    x=alt.X('Priority', sort='-y'),
    # Use 'Delay_Rate_Percent' on the y-axis
    y=alt.Y('Delay_Rate_Percent', title='Delay Rate (%)'),
    # Add tooltips to show details on hover
    tooltip=['Priority', alt.Tooltip('Delay_Rate_Percent', format='.1f')]
).properties(
    title = 'Delay Rate by Order Priority',
    width=400, height=250 
).interactive()

In [166]:
priority_chart

alt.Chart(...)

In [167]:
carrier_delay_df = merged_df.groupby('Carrier')['is_delayed'].mean().reset_index()

In [168]:
carrier_delay_df = carrier_delay_df.rename(columns={'is_delayed': 'Delay_Rate'})
carrier_delay_df['Delay_Rate_Percent'] = carrier_delay_df['Delay_Rate'] * 100

In [169]:
print(carrier_delay_df.sort_values(by='Delay_Rate', ascending=False).to_markdown(index=False, floatfmt=".2f"))

| Carrier         |   Delay_Rate |   Delay_Rate_Percent |
|:----------------|-------------:|---------------------:|
| GlobalTransit   |         0.64 |                63.64 |
| ReliableExpress |         0.53 |                52.50 |
| SpeedyLogistics |         0.51 |                51.22 |
| EcoDeliver      |         0.38 |                37.50 |
| QuickShip       |         0.26 |                25.81 |


In [170]:
carrier_chart = alt.Chart(carrier_delay_df).mark_bar().encode(
    # Use 'Carrier' on the x-axis, sort by delay rate (descending)
    x=alt.X('Carrier', sort='-y'),
    # Use 'Delay_Rate_Percent' on the y-axis
    y=alt.Y('Delay_Rate_Percent', title='Delay Rate (%)'),
    # Add tooltips to show details on hover
    tooltip=['Carrier', alt.Tooltip('Delay_Rate_Percent', format='.1f')]
).properties(
    title='Delay Rate by Carrier',
    width = 400, height = 250
).interactive()

In [171]:
carrier_chart

alt.Chart(...)

In [172]:
print("Weather Impact Value Counts:")
print(merged_df['Weather_Impact'].value_counts())
print("\n") 

Weather Impact Value Counts:
Weather_Impact
Normal Weather    106
Light_Rain         24
Heavy_Rain         14
Fog                 6
Name: count, dtype: int64




In [173]:
weather_delay_df = merged_df.groupby('Weather_Impact')['is_delayed'].mean().reset_index()

In [174]:
weather_delay_df = weather_delay_df.rename(columns={'is_delayed': 'Delay_Rate'})
weather_delay_df['Delay_Rate_Percent'] = weather_delay_df['Delay_Rate'] * 100

In [175]:
print(weather_delay_df.to_markdown(index=False, floatfmt=".2f"))

| Weather_Impact   |   Delay_Rate |   Delay_Rate_Percent |
|:-----------------|-------------:|---------------------:|
| Fog              |         0.17 |                16.67 |
| Heavy_Rain       |         0.50 |                50.00 |
| Light_Rain       |         0.38 |                37.50 |
| Normal Weather   |         0.50 |                50.00 |


In [176]:
weather_chart = alt.Chart(weather_delay_df).mark_bar().encode(
    # Use 'Weather_Impact' on the x-axis, sort by delay rate (descending)
    x=alt.X('Weather_Impact', sort='-y'),
    # Use 'Delay_Rate_Percent' on the y-axis
    y=alt.Y('Delay_Rate_Percent', title='Delay Rate (%)'),
    # Add tooltips to show details on hover
    tooltip=['Weather_Impact', alt.Tooltip('Delay_Rate_Percent', format='.1f')]
).properties(
    title='Delay Rate by Weather Impact',
    width = 400, height = 250
).interactive()

In [177]:
weather_chart

alt.Chart(...)

In [178]:
print("Special Handling Value Counts:")
print(merged_df['Special_Handling'].value_counts())

Special Handling Value Counts:
Special_Handling
Normal Handling           115
Fragile                    18
Temperature_Controlled     10
Hazmat                      7
Name: count, dtype: int64


In [179]:
handling_delay_df = merged_df.groupby('Special_Handling')['is_delayed'].mean().reset_index()

In [180]:
handling_delay_df = handling_delay_df.rename(columns={'is_delayed': 'Delay_Rate'})
handling_delay_df['Delay_Rate_Percent'] = handling_delay_df['Delay_Rate'] * 100

In [181]:
print(handling_delay_df.to_markdown(index=False, floatfmt=".2f"))

| Special_Handling       |   Delay_Rate |   Delay_Rate_Percent |
|:-----------------------|-------------:|---------------------:|
| Fragile                |         0.39 |                38.89 |
| Hazmat                 |         0.71 |                71.43 |
| Normal Handling        |         0.46 |                46.09 |
| Temperature_Controlled |         0.50 |                50.00 |


In [182]:
handling_chart = alt.Chart(handling_delay_df).mark_bar().encode(
    # Use 'Special_Handling' on the x-axis, sort by delay rate (descending)
    x=alt.X('Special_Handling', sort='-y'),
    # Use 'Delay_Rate_Percent' on the y-axis
    y=alt.Y('Delay_Rate_Percent', title='Delay Rate (%)'),
    # Add tooltips to show details on hover
    tooltip=['Special_Handling', alt.Tooltip('Delay_Rate_Percent', format='.1f')]
).properties(
    title='Delay Rate by Special Handling',
    width = 400, height = 250
).interactive()

In [183]:
handling_chart

alt.Chart(...)

In [196]:
route_traffic_df = merged_df.groupby('Route')['Traffic_Delay_Minutes'].mean().reset_index()
route_traffic_df.columns = ['Route', 'Average_Traffic_Delay_Minutes']

In [197]:
route_traffic_df = route_traffic_df.sort_values(by='Average_Traffic_Delay_Minutes', ascending=False)

In [200]:
#Top 10 worst routes
top_10_routes_df = route_traffic_df.head(10)

In [201]:
print("Top 10 Routes with Highest Average Traffic Delays:\n")
print(top_10_routes_df.to_markdown(index=False, floatfmt='.1f'))

Top 10 Routes with Highest Average Traffic Delays:

| Route               |   Average_Traffic_Delay_Minutes |
|:--------------------|--------------------------------:|
| Mumbai-Chennai      |                           114.0 |
| Pune-Mumbai         |                           108.0 |
| Hyderabad-Chennai   |                           106.0 |
| Hyderabad-Bangalore |                           102.0 |
| Pune-Hyderabad      |                           101.0 |
| Bangalore-Chennai   |                           100.0 |
| Chennai-Kolkata     |                            83.3 |
| Mumbai-Kolkata      |                            82.5 |
| Mumbai-Pune         |                            81.8 |
| Ahmedabad-Bangalore |                            81.0 |


In [184]:
merged_df['Current_Stock_Units'] = merged_df['Current_Stock_Units'].fillna(0)
merged_df['Reorder_Level'] = merged_df['Reorder_Level'].fillna(0)

In [185]:
merged_df['Stock_vs_Reorder'] = merged_df['Current_Stock_Units'] - merged_df['Reorder_Level']

In [186]:
merged_df['Is_Below_Reorder'] = (merged_df['Stock_vs_Reorder'] < 0).astype(int)

In [187]:
stock_delay_df = merged_df.groupby('Is_Below_Reorder')['is_delayed'].mean().reset_index()

In [188]:
stock_delay_df = stock_delay_df.rename(columns={'is_delayed': 'Delay_Rate', 'Is_Below_Reorder': 'Stock_Below_Reorder_Level'})
stock_delay_df['Delay_Rate_Percent'] = stock_delay_df['Delay_Rate'] * 100

In [189]:
stock_delay_df['Stock_Below_Reorder_Level'] = stock_delay_df['Stock_Below_Reorder_Level'].map({0: 'No (Stock OK)', 1: 'Yes (Stock Low)'})

In [190]:
print(stock_delay_df.to_markdown(index=False, floatfmt=".2f"))

| Stock_Below_Reorder_Level   |   Delay_Rate |   Delay_Rate_Percent |
|:----------------------------|-------------:|---------------------:|
| No (Stock OK)               |         0.49 |                48.74 |
| Yes (Stock Low)             |         0.39 |                38.71 |


In [191]:
stock_chart = alt.Chart(stock_delay_df).mark_bar().encode(
    # Use our readable labels on the x-axis, sort by delay rate
    x=alt.X('Stock_Below_Reorder_Level', sort='-y'),
    # Use 'Delay_Rate_Percent' on the y-axis
    y=alt.Y('Delay_Rate_Percent', title='Delay Rate (%)'),
    # Add tooltips to show details on hover
    tooltip=['Stock_Below_Reorder_Level', alt.Tooltip('Delay_Rate_Percent', format='.1f')]
).properties(
    title='Delay Rate for Orders with Low Inventory',
    width = 400, height = 250
).interactive()

In [192]:
stock_chart

alt.Chart(...)